In [6]:
!pip install requests

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [7]:
## Imports
import urllib
import json
import time
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import urllib

In [8]:
## Set-up values
baseurl="https://api.nytimes.com/svc/search/v2/articlesearch.json?"
apikey="JQwerlg2vtarptmt8m7zyH46QzVt6EKi"

In [9]:
## Creating Lists (Final Lists)
date_params = ["2020-01-01", "2020-03-31", "2020-04-01", "2020-06-30", "2020-07-01", "2020-09-30", "2020-10-01", "2020-12-31",
               "2021-01-01", "2021-03-31", "2021-04-01", "2021-06-30", "2021-07-01", "2021-09-30", "2021-10-01", "2021-12-31",
               "2022-01-01", "2022-03-31", "2022-04-01", "2022-06-30", "2022-07-01", "2022-09-30", "2022-10-01", "2022-12-31",
               "2023-01-01", "2023-03-31", "2023-04-01", "2023-06-30", "2023-07-01", "2023-09-30", "2023-10-01", "2023-12-31",
               "2024-01-01", "2024-03-31", "2024-04-01", "2024-06-30", "2024-07-01", "2024-09-30", "2024-10-01", "2024-12-31"]

## Split into 5 so it can be run in separate chunks
states1 = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "District of Columbia", "Florida"]
states2 = ["Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine"]
states3 = ["Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire"]
states4 = ["New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island"]
states5 = ["South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]

years = [2020, 2021, 2022, 2023, 2024]
quarters = ["Q1", "", "Q2", "", "Q3", "", "Q4", ""]
year_strings = ["2020", "2021", "2022", "2023", "2024"]
quarter_dates = ["-01-01", "-03-31", "-04-01", "-06-30", "-07-01", "-09-30", "-10-01", "-12-31"]
    

In [10]:
url_dict = {}

# Initialize DataFrame
df = pd.DataFrame({"State": [], "Year": [], "Quarter": [], "Abstract": [], "Web URL": []})

for state in states1:
    for year in range(0, len(year_strings)):
        for quarter_iter in range(0, len(quarter_dates), 2):
            # Define API query parameters
            pa = {"api-key": apikey, 
                  "q": "telemedicine" + state, 
                  "begin_date": year_strings[year] + quarter_dates[quarter_iter], 
                  "end_date": year_strings[year] + quarter_dates[quarter_iter + 1], 
                  "page": 1}
            
            # Generate the URL to check
            url2check = baseurl + urllib.parse.urlencode(pa)
            
            # Request the data
            result = urllib.request.urlopen(url2check).read()
            resd = json.loads(result)
            
            # Print out the full response to inspect available fields
            # print(json.dumps(resd, indent=4))  # Uncomment for debugging
            
            # Get the number of hits
            if len(resd['response']['docs']) < 10:
                hit_num = len(resd['response']['docs'])
            else:
                hit_num = 10

            if hit_num == 0:
                new_empty_row = [state, years[year], quarters[quarter_iter], "", ""]
                print(new_empty_row)
                df.loc[len(df.index)] = new_empty_row
            else:
                for i in range(0, hit_num):
                    # Inspect if full text or another field is available
                    if "full_text" in resd['response']['docs'][i]:
                        full_text = resd['response']['docs'][i]["full_text"]
                    elif "content" in resd['response']['docs'][i]:
                        full_text = resd['response']['docs'][i]["content"]
                    elif "body" in resd['response']['docs'][i]:
                        full_text = resd['response']['docs'][i]["body"]
                    else:
                        # Fallback to abstract if no full text is found
                        full_text = resd['response']['docs'][i].get("abstract", "No full text available")
                    
                    # Store the web_url or other similar field in the dictionary
                    if "web_url" in resd['response']['docs'][i]:
                        web_url = resd['response']['docs'][i]["web_url"]
                    else:
                        web_url = "No URL available"
                    
                    # Add the URL to the dictionary, using a tuple (state, year, quarter) as the key
                    key = (state, years[year], quarters[quarter_iter])
                    if key not in url_dict:
                        url_dict[key] = []
                    url_dict[key].append(web_url)
                    
                    # Create a new row with the full article content and the URL
                    new_row = [state, years[year], quarters[quarter_iter], full_text, web_url]
                    print(new_row)
                    df.loc[len(df.index)] = new_row
            
            # Respect API rate limit (12 seconds)
            time.sleep(12)

# After the loop, print the DataFrame to check the result
print("DataFrame with URLs:")
print(df)

print("Done")

HTTPError: HTTP Error 429: Too Many Requests

In [ ]:
df.head()

In [ ]:
url = df['Web URL']

In [ ]:
import requests
from bs4 import BeautifulSoup
import time


base_url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?'

# List to store all reviews
all_reviews = []
page = 1  # Starting page

while True:
    # Construct the URL for each page
    url = f"{base_url}?page={page}"
    print(f"Fetching reviews from: {url}")
    
    # Send a GET request to fetch the raw HTML content
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find all review elements by their HTML tag and class name
        reviews = soup.find_all(class_='nytapp-vi-interactive')
        
        # Check if there are no reviews on the current page
        if not reviews:
            print("No more reviews found.")
            break  # Exit the loop if no reviews are found
        
        # Extract text from each review element
        for review in reviews:
            all_reviews.append(review.get_text(strip=True))
        
        # Increment the page number for the next request
        page += 1
        time.sleep(1)  # Pause to be polite to the server
    
    else:
        print(f"Failed to retrieve page. Status code: {response.status_code}")
        break

# Print all reviews
for i, review in enumerate(all_reviews, start=1):
    print(f"Page {i}: {review}\n")

In [ ]:
corpus = all_reviews

In [ ]:
corpus

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
 
nltk.download('stopwords')
nltk.download('punkt')

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

corpus_cleaned = []

for text in corpus:
    # Seperate text into individual words
    tokens = word_tokenize(text)
    
    # Remove the punctuations and numbers
    tokens = [word for word in tokens if word.isalpha()]

    # Lower the tokens
    tokens = [word.lower() for word in tokens]

    # Remove stopword
    tokens = [word for word in tokens if not word in stopwords.words("english")]

    # Stem the tokens
    ps = PorterStemmer()
    tokens = [ps.stem(w) for w in tokens]
    
    text_cleaned = " ".join(tokens)

    corpus_cleaned.append(text_cleaned)

corpus_cleaned

In [ ]:
# import nltk vader library
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
    
# initiate an analyzer
sia = SentimentIntensityAnalyzer()

# iterate through each sentence in corpus
for sentence in corpus:
    
    print(sentence)
    
    # analyze the sentiment. ss is a dictionary
    ss = sia.polarity_scores(sentence)
    
    # output each sentiment score (neg, neu, pos, compound) in ss
    for k in ss:
        print('{0}: {1}, '.format(k, ss[k]))
    
    # print an empty line as seperator
    print('\n')

In [ ]:
'''
import requests
from bs4 import BeautifulSoup
import time

# List to store all reviews
all_reviews = []

# Iterate through each URL in url_dict
for key, urls in url_dict.items():
    state, year, quarter = key
    print(f"Fetching reviews for {state} {year} {quarter}")
    
    for base_url in urls:
        page = 1  # Start from the first page for each URL

        while True:
            # Construct the URL for the current page
            url = f"{base_url}?page={page}"
            print(f"Fetching text from: {url}")
            
            # Send a GET request to fetch the raw HTML content
            response = requests.get(url)
            
            # Check if the request was successful
            if response.status_code == 200:
                # Parse the HTML content using BeautifulSoup
                soup = BeautifulSoup(response.text, 'html.parser')
                
                # Find all review elements by their HTML tag and class name
                # You need to change this based on the structure of the webpage you're scraping.
                reviews = soup.find_all(class_='review-class-name')  # Modify the class name accordingly
                
                # If no reviews are found, break the loop
                if not reviews:
                    print(f"No more text found for {base_url}. Moving to next URL.")
                    break  # Exit the loop if no reviews are found
                
                # Extract text from each review element
                for review in reviews:
                    all_reviews.append(review.get_text(strip=True))
                
                # Increment the page number for the next request
                page += 1
                time.sleep(1)  # Pause to be polite to the server
            
            else:
                print(f"Failed to retrieve page. Status code: {response.status_code} for URL {url}")
                break  # Exit the loop if request fails
    
    # After fetching reviews for a specific state, year, and quarter, print the results
    print(f"Text for {state} {year} {quarter}:")
    for i, review in enumerate(all_reviews, start=1):
        print(f"Text {i}: {review}\n")
'''

In [ ]:
corpus = df['Abstract']

In [ ]:
corpus